I want to predict if an NBA team will make the playoffs. I chose this for my first Machine Learning project because I already know which variables increase the chances of making the playoffs. I am also initerested in the power of the variables when creating the models.

In [415]:
import matplotlib.pyplot as plt  # To visualize
from sklearn.model_selection import GridSearchCV
import matplotlib
matplotlib.use("Qt5Agg")
import pandas as pd
import numpy as np
import os
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.tree import export_graphviz
from sklearn import tree
import numpy as np
from sklearn import metrics
pd.options.mode.chained_assignment = None  # default='warn'

First, I am going to initialize some lists and variables that I will use for iteration purposes or throughout the program

In [348]:
# Variables I'll need throughout
path = "/Users/matthewmacrides/Python and R/Github/Machine Learning/NbaPlayoffs"

sep = r'/'

seasons = ['18-19', '21-22']

models = ['Decision Tree', 'Random Forest']

I decided to make a for-loop in the case that I add more years of data in the future. All I will need to do is save the .csv file and update the seasons list.

In [350]:
def readData():
     # Create dataframes that will store all the years data
     Nba = pd.DataFrame()

     for season in seasons:
          Nba_ = pd.read_csv(path+sep+season+'.csv', sep=',', decimal='.')
          
          # Create a season column at the end of each file
          Nba_['Season'] = season

          # Not needed because we have win percentage and PD
          Nba_ = Nba_.drop(columns=['Wins', 'Losses', 'PS', 'PA'])
          
          # Concatenate files together for each category
          Nba = pd.concat([Nba,Nba_], ignore_index=True)

          Nba.dropna(how='any', inplace=True)

          Teams = Nba[['Team', 'Season']].copy()

          
     return Nba, Teams

I was interested in the correlations across variables. To no surprise, Wins, Losses, Win Percentage, PD, and Standardized PD had strong correlations to Playoffs. I am going to include these variables in my first iteration of the modeliing. Then I am going to remove them to capture the change in accuracy. I am assuming my models will have an accuracy score close to 1 during my first iteration since the variables I mentioned are directly embedded into a team's playoff chances.

In [351]:
Nba, Teams = readData()
Nba.corr().style.background_gradient(cmap="Blues")

,Seed,Win Percentage,PD,FG%,3P%,FT%,TRB,AST,STL,BLK,TOV,PF,Year,Playoff,QF,SF,C
Seed,1.000000,-0.957713,-0.922434,-0.330752,-0.346985,-0.306488,-0.441456,-0.394662,-0.227805,-0.157820,0.286439,0.131091,-0.000000,-0.850561,-0.767649,-0.533357,-0.236391
Win Percentage,-0.957713,1.000000,0.963583,0.308621,0.412242,0.255040,0.460130,0.432439,0.220368,0.201503,-0.323994,-0.136303,-0.000118,0.832284,0.682190,0.460361,0.231731
PD,-0.922434,0.963583,1.000000,0.263779,0.417414,0.202691,0.506755,0.459776,0.217250,0.279725,-0.306032,-0.174153,0.000696,0.779001,0.638840,0.471304,0.223131
FG%,-0.330752,0.308621,0.263779,1.000000,-0.449159,0.263556,0.037558,0.383117,-0.017983,0.097278,-0.028515,-0.009798,-0.143051,0.277632,0.314114,0.248379,0.118633
3P%,-0.346985,0.412242,0.417414,-0.449159,1.000000,0.128708,0.114698,0.201550,0.010749,0.002822,-0.181420,-0.155778,0.103298,0.312974,0.090442,0.091942,0.050906
FT%,-0.306488,0.255040,0.202691,0.263556,0.128708,1.000000,-0.274314,0.010310,-0.233418,-0.149210,-0.401102,-0.240016,0.135684,0.324486,0.340267,0.311588,0.097333
TRB,-0.441456,0.460130,0.506755,0.037558,0.114698,-0.274314,1.000000,0.278683,0.059059,0.360914,0.032237,-0.017047,-0.194055,0.311180,0.299456,0.235744,0.054053
AST,-0.394662,0.432439,0.459776,0.383117,0.201550,0.010310,0.278683,1.000000,0.278905,0.315405,0.152602,-0.018332,0.020278,0.207004,0.269287,0.198499,0.160282
STL,-0.227805,0.220368,0.217250,-0.017983,0.010749,-0.233418,0.059059,0.278905,1.000000,0.317705,0.093445,0.334329,0.008376,0.081432,0.190366,-0.051747,0.214654
BLK,-0.157820,0.201503,0.279725,0.097278,0.002822,-0.149210,0.360914,0.315405,0.317705,1.000000,0.104905,0.165683,-0.160995,0.096496,0.153477,0.099828,0.017586


Saving a copy of the Team column so we can join with our test data after the analysis. Also creating a copy of the Nba raw data file in the case that I model whether or not a team will make the quarterfinals. Lastly, I remove all string columns and additional columns that aren't important for this analysis


In [352]:
Nba = Nba.drop(columns=['Team', 'Year', 'QF', 'SF', 'F', 'C'])

Here, I define a function called 'modeling' to split test and training data. I will chose Playoff as my y value. Playoff will be denoted as a '1' if the team made it and a '0' otherwise. Then I iterated through different models and recorded the accuracy for all of them.

In [353]:
y = Nba[["Playoff", "Season"]]
x = Nba.drop('Playoff', axis=1)
Round = "Playoff"

In [ ]:
scaler = StandardScaler()

        scaled_X_train = scaler.fit_transform(X_train)
        scaled_X_test = scaler.transform(X_test)


CHANGE HYPERPARAMETERS

In [429]:
def modeling(models, x, y, Round):
    # Create x and y datasets with the target variable 'Playoff' in mind. 0 denotes a team does not make the playoffs. 1 denotes that a team does make the playoffs.
    for season in seasons:

        # Separating training and testing data
        season_test = season

        x_train = x[x['Season']!=season_test]
        x_test = x[x['Season']==season_test]
        y_train = y[y['Season']!=season_test]
        y_test = y[y['Season']==season_test]

    datasets = [x_train, x_test, y_train, y_test]
    for dataset in datasets:
        dataset.drop(['Season'], axis=1, inplace=True)

    feature_list = list(x_train.columns)
    
    metric = pd.DataFrame()
    compare = pd.DataFrame()
    features = pd.DataFrame()
    
    for model in models:
            if model == 'Decision Treee':
                model_ = DecisionTreeClassifier() 
            elif model=='Random Forest':
                model_ = RandomForestClassifier()
                param_grid = { "n_estimators": [100, 200, 300],
                                "max_depth": [5, 10, 15],
                                "min_samples_split": [2, 5, 10]}
                grid_search = GridSearchCV(model_, param_grid, cv=5, scoring="accuracy")
                grid_search.fit(x_train, y_train.values.ravel())
            
            y_pred = grid_search.predict(x_test)

            #model_.fit(x_train, y_train.values.ravel())
            #y_pred = model_.predict(x_test)
            
            # Store the accuracy in a dataframe. Concatenate for each accuracy score and record the model that received the result
            accuracy = metrics.accuracy_score(y_test, y_pred)
            data = {'Accuracy Score': [accuracy], 'Model': [model]}
            accuracy = pd.DataFrame(data)
            metric = pd.concat([metric,accuracy], ignore_index=True)

            # Find probabilities
            probability = model_.predict_proba(x_test)
            probability = probability[:, 1]

            # Comparing real vs. expected playoffs
            if Round == "Playoff": compare['Playoff'] = y_test
            if Round == "Playoff": compare['Pred Playoff ' + model] = y_pred
            if Round == "QF": compare['QF'] = y_test
            if Round == "QF": compare['Pred QF ' + model] = y_pred
            if Round == "SF": compare['SF'] = y_test
            if Round == "SF": compare['Pred SF ' + model] = y_pred
            if Round == "C": compare['C'] = y_test
            if Round == "C": compare['Pred C ' + model] = y_pred
            compare['Probability ' + model] = probability

            # Store the variable importances
            # Get numerical feature importances
            importances = list(model_.feature_importances_)
            # List of tuples with variable and importance
            feature_importances = [(feature, round(importance, 2)) for feature, importance in zip(feature_list, importances)]
            # Sort the feature importances by most important first
            feature_importances = sorted(feature_importances, key = lambda x: x[1], reverse = True)
            features_ = pd.DataFrame(feature_importances, columns=['Variables', 'Importance'])
            features_['Model'] = model
            features_ = features_.head(5)
            features = pd.concat([features, features_], ignore_index=True)
            
    return metric, compare, features

As we can see, the models are almost 100% accurate when predicting the playoff teams. The only reason they aren't 100% is because there are years that non-playoff teams in the west have a better record than playoff teams in the east and vice versa.

In [430]:
metric, compare, features = modeling(models, x, y, Round)
metric

UnboundLocalError: local variable 'grid_search' referenced before assignment

In [356]:
features

,Variables,Importance,Model
0,Win Percentage,1.00,Decision Tree
1,Seed,0.00,Decision Tree
2,PD,0.00,Decision Tree
3,FG%,0.00,Decision Tree
4,3P%,0.00,Decision Tree
5,Seed,0.32,Random Forest
6,PD,0.27,Random Forest
7,Win Percentage,0.22,Random Forest
8,3P%,0.06,Random Forest
9,FG%,0.03,Random Forest


Thiis is just showing the team, if they made the playoffs, and what the models predicted.

In [357]:
compare = pd.merge(Teams, compare, left_index=True, right_index=True)
compare

,Team,Season,Playoff,Pred Playoff Decision Tree,Probability Decision Tree,Pred Playoff Random Forest,Probability Random Forest
30,Phoenix Suns,21-22,1.0,1.0,1.0,1.0,0.98
31,Memphis Grizzlies,21-22,1.0,1.0,1.0,1.0,0.98
32,Golden State Warriors,21-22,1.0,1.0,1.0,1.0,0.97
33,Miami Heat,21-22,1.0,1.0,1.0,1.0,0.97
34,Dallas Mavericks,21-22,1.0,1.0,1.0,1.0,0.94
35,Boston Celtics,21-22,1.0,1.0,1.0,1.0,1.00
36,Milwaukee Bucks,21-22,1.0,1.0,1.0,1.0,0.99
37,Philadelphia 76ers,21-22,1.0,1.0,1.0,1.0,0.98
38,Utah Jazz,21-22,1.0,1.0,1.0,1.0,0.95
39,Denver Nuggets,21-22,1.0,1.0,1.0,1.0,0.96


In [358]:
NbaSmall, Teams = readData()
NbaSmall = NbaSmall.drop(columns=['Win Percentage', 'PD', 'Seed', 'Team', 'Year', 'QF', 'SF', 'F', 'C'])
y = NbaSmall[["Playoff", "Season"]]
x = NbaSmall.drop('Playoff', axis=1)

In [311]:
metric, compare, features = modeling(models, x, y, Round)
metric

,Accuracy Score,Model
0,0.733333,Decision Tree
1,0.800000,Random Forest


In [312]:
features

,Variables,Importance,Model
0,3P%,0.45,Decision Tree
1,TOV,0.22,Decision Tree
2,BLK,0.14,Decision Tree
3,STL,0.10,Decision Tree
4,FT%,0.09,Decision Tree
5,3P%,0.28,Random Forest
6,FG%,0.14,Random Forest
7,TRB,0.12,Random Forest
8,FT%,0.10,Random Forest
9,BLK,0.08,Random Forest


In [313]:
compare = pd.merge(Teams, compare, left_index=True, right_index=True)
compare

,Team,Season,Playoff,Pred Playoff Decision Tree,Probability Decision Tree,Pred Playoff Random Forest,Probability Random Forest
30,Phoenix Suns,21-22,1.0,0.0,0.0,1.0,0.77
31,Memphis Grizzlies,21-22,1.0,1.0,1.0,1.0,0.70
32,Golden State Warriors,21-22,1.0,1.0,1.0,1.0,0.64
33,Miami Heat,21-22,1.0,1.0,1.0,1.0,0.73
34,Dallas Mavericks,21-22,1.0,0.0,0.0,0.0,0.39
35,Boston Celtics,21-22,1.0,1.0,1.0,1.0,0.85
36,Milwaukee Bucks,21-22,1.0,0.0,0.0,1.0,0.78
37,Philadelphia 76ers,21-22,1.0,1.0,1.0,1.0,0.80
38,Utah Jazz,21-22,1.0,1.0,1.0,1.0,0.60
39,Denver Nuggets,21-22,1.0,1.0,1.0,1.0,0.70


In [36]:
import pydot
# Limit depth of tree to 3 levels
rf_small = RandomForestClassifier(n_estimators=10, max_depth = 3)
rf_small.fit(x_train, y_train.values.ravel())
# Extract the small tree
tree_small = rf_small.estimators_[5]
# Save the tree as a png image
export_graphviz(tree_small, out_file = 'small_tree.dot', feature_names = feature_list, rounded = True, precision = 1)
(graph, ) = pydot.graph_from_dot_file('small_tree.dot')
graph.write_png('small_tree.png')

In [383]:
Nba, Teams = readData()
NbaQF = Nba[Nba['Playoff'] == 1]
NbaQF = NbaQF.drop(columns=['Team', 'Year', 'Playoff', 'SF', 'F', 'C'])
y = NbaQF[["QF", "Season"]]
x = NbaQF.drop('QF', axis=1)
Round = 'QF'

In [384]:
metric, compare, features = modeling(models, x, y, Round)
metric

,Accuracy Score,Model
0,1.0,Decision Tree
1,1.0,Random Forest


In [361]:
features

,Variables,Importance,Model
0,Seed,1.00,Decision Tree
1,Win Percentage,0.00,Decision Tree
2,PD,0.00,Decision Tree
3,FG%,0.00,Decision Tree
4,3P%,0.00,Decision Tree
5,Seed,0.33,Random Forest
6,Win Percentage,0.17,Random Forest
7,PD,0.12,Random Forest
8,FT%,0.09,Random Forest
9,AST,0.07,Random Forest


In [385]:
compare = pd.merge(Teams, compare, left_index=True, right_index=True)
compare

,Team,Season,QF,Pred QF Decision Tree,Probability Decision Tree,Pred QF Random Forest,Probability Random Forest
30,Phoenix Suns,21-22,1.0,1.0,1.0,1.0,0.90
31,Memphis Grizzlies,21-22,1.0,1.0,1.0,1.0,0.86
32,Golden State Warriors,21-22,1.0,1.0,1.0,1.0,0.82
33,Miami Heat,21-22,1.0,1.0,1.0,1.0,0.85
34,Dallas Mavericks,21-22,1.0,1.0,1.0,1.0,0.72
35,Boston Celtics,21-22,1.0,1.0,1.0,1.0,0.86
36,Milwaukee Bucks,21-22,1.0,1.0,1.0,1.0,0.72
37,Philadelphia 76ers,21-22,1.0,1.0,1.0,1.0,0.77
38,Utah Jazz,21-22,0.0,0.0,0.0,0.0,0.27
39,Denver Nuggets,21-22,0.0,0.0,0.0,0.0,0.21


In [402]:
Nba, Teams = readData()
NbaSF = Nba[Nba['QF'] == 1]
NbaSF = NbaSF.drop(columns=['Team', 'Year', 'Playoff', 'QF', 'F', 'C'])
y = NbaSF[["SF", "Season"]]
x = NbaSF.drop('SF', axis=1)
Round = 'SF'

In [403]:
metric, compare, features = modeling(models, x, y, Round)
metric

,Accuracy Score,Model
0,0.375,Decision Tree
1,0.500,Random Forest


In [404]:
compare = pd.merge(Teams, compare, left_index=True, right_index=True)
compare

,Team,Season,SF,Pred SF Decision Tree,Probability Decision Tree,Pred SF Random Forest,Probability Random Forest
30,Phoenix Suns,21-22,0.0,1.0,1.0,1.0,0.73
31,Memphis Grizzlies,21-22,0.0,1.0,1.0,1.0,0.58
32,Golden State Warriors,21-22,1.0,0.0,0.0,0.0,0.38
33,Miami Heat,21-22,1.0,1.0,1.0,1.0,0.54
34,Dallas Mavericks,21-22,1.0,0.0,0.0,0.0,0.17
35,Boston Celtics,21-22,1.0,1.0,1.0,1.0,0.58
36,Milwaukee Bucks,21-22,0.0,0.0,0.0,0.0,0.42
37,Philadelphia 76ers,21-22,0.0,1.0,1.0,0.0,0.26


In [393]:
features

,Variables,Importance,Model
0,FG%,0.60,Decision Tree
1,TOV,0.40,Decision Tree
2,Seed,0.00,Decision Tree
3,Win Percentage,0.00,Decision Tree
4,PD,0.00,Decision Tree
5,FG%,0.17,Random Forest
6,Win Percentage,0.11,Random Forest
7,PD,0.11,Random Forest
8,TOV,0.11,Random Forest
9,TRB,0.09,Random Forest


In [420]:
NbaC, Teams = readData()
NbaC = NbaC.drop(columns=['Team', 'Year', 'Playoff', 'QF', 'F'])
y = NbaC[["C", "Season"]]
x = NbaC.drop('C', axis=1)
Round = 'C'

In [421]:
metric, compare, features = modeling(models, x, y, Round)
metric

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/model_selection/_split.py:684: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:686: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:686: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/model_selection/_validation

Best hyperparameters:  {'max_depth': 5, 'min_samples_split': 2, 'n_estimators': 100}
Best test score:  0.9666666666666666


,Accuracy Score,Model
0,0.966667,Decision Tree
1,0.966667,Random Forest


In [422]:
compare = pd.merge(Teams, compare, left_index=True, right_index=True)
compare

,Team,Season,C,Pred C Decision Tree,Probability Decision Tree,Pred C Random Forest,Probability Random Forest
30,Phoenix Suns,21-22,0.0,0.0,0.0,0.0,0.31
31,Memphis Grizzlies,21-22,0.0,0.0,0.0,0.0,0.13
32,Golden State Warriors,21-22,1.0,0.0,0.0,0.0,0.13
33,Miami Heat,21-22,0.0,0.0,0.0,0.0,0.19
34,Dallas Mavericks,21-22,0.0,0.0,0.0,0.0,0.03
35,Boston Celtics,21-22,0.0,0.0,0.0,0.0,0.14
36,Milwaukee Bucks,21-22,0.0,0.0,0.0,0.0,0.03
37,Philadelphia 76ers,21-22,0.0,0.0,0.0,0.0,0.05
38,Utah Jazz,21-22,0.0,0.0,0.0,0.0,0.08
39,Denver Nuggets,21-22,0.0,0.0,0.0,0.0,0.03
